In [ ]:
import os
import numpy as np
import random
import tifffile as tiff

def simulate_missing_bscans(volume: np.ndarray, missing_fraction: float = 0.05, block_removal: bool = True, block_size_range=(3, 10)):
    """
    Simulate missing B-scans in an OCTA volume.
    
    Args:
        volume (np.ndarray): Input 3D volume of shape (D, H, W)
        missing_fraction (float): Fraction of slices to remove (e.g., 0.05 = 5%)
        block_removal (bool): Whether to remove contiguous blocks instead of random single slices
        block_size_range (tuple): If block_removal=True, size range of the blocks to remove
        
    Returns:
        corrupted_volume (np.ndarray): Same shape as input, with missing bscans zeroed out
        mask (np.ndarray): Binary mask of missing indices (1 = missing, 0 = present)
        missing_indices (list): List of indices where bscans were removed
    """
    D, H, W = volume.shape
    num_missing = int(D * missing_fraction)
    corrupted_volume = volume.copy()
    mask = np.zeros(D, dtype=np.uint8)
    missing_indices = set()

    if block_removal:
        while len(missing_indices) < num_missing:
            block_size = random.randint(*block_size_range)
            start_idx = random.randint(0, D - block_size)
            block_indices = list(range(start_idx, start_idx + block_size))
            if len(missing_indices.union(block_indices)) <= num_missing:
                for idx in block_indices:
                    corrupted_volume[idx] = 0
                    mask[idx] = 1
                missing_indices.update(block_indices)
    else:
        all_indices = list(range(D))
        random.shuffle(all_indices)
        selected = all_indices[:num_missing]
        for idx in selected:
            corrupted_volume[idx] = 0
            mask[idx] = 1
        missing_indices = selected

    return corrupted_volume, mask, sorted(missing_indices)

# input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.1_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD.tif"
# input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.2_OCT_uint16_Cropped_VolumeSplit_2_Reflected_RegSeq_seqSVD.tif"
# input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.4_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD.tif"
# input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD.tif"
input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/28percent_2to8sizeblock/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_gt_v2.tif"

# Load volume
volume = tiff.imread(input_path)  # shape: (1000, 1280, 400)

print("Volume shape:", volume.shape)

# Simulate missing bscans
corrupted_volume, mask, missing = simulate_missing_bscans(volume, missing_fraction=0.2, block_removal=True, block_size_range=(2, 6))

# Generate output paths
base_dir = os.path.dirname(input_path)
base_name = os.path.splitext(os.path.basename(input_path))[0]
corrupted_path = os.path.join(base_dir, f"{base_name}_corrupted.tif")
mask_path = os.path.join(base_dir, f"{base_name}_mask.tif")


# Save corrupted volume as TIF
tiff.imwrite(corrupted_path, corrupted_volume.astype(np.uint16), imagej=True)

# Save mask as TIF (binary, 1 = missing, 0 = present)
# We'll expand dims to (D, 1, 1) and tile to match image size for visualization
mask_volume = np.tile(mask[:, None, None], (1, volume.shape[1], volume.shape[2]))
tiff.imwrite(mask_path, mask_volume.astype(np.uint8), imagej=True)

print(f"Corrupted volume saved to: {corrupted_path}")
print(f"Mask saved to: {mask_path}")

# Optionally, log missing indices for inspection
print(f"Removed {len(missing)} B-scans at indices: {missing[:10]}...")

Volume shape: (1000, 350, 400)
Corrupted volume saved to: /media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/28percent_2to8sizeblock/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_gt_v2_corrupted.tif
Mask saved to: /media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/28percent_2to8sizeblock/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_gt_v2_mask.tif
Removed 200 B-scans at indices: [23, 24, 25, 28, 29, 30, 31, 39, 40, 55]...
